In [16]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler, MultiLabelBinarizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity,euclidean_distances

####  1.Data Preprocessing:


In [2]:
df=pd.read_csv('/anime.csv')
df.head()

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266


In [3]:
df.shape

(12294, 7)

In [4]:
df.duplicated().any()

False

In [17]:
len(df.anime_id.unique())

12294

In [18]:
len(df.name.unique())

12292

In [20]:
len(df.type.unique())

7

In [21]:
df.rating.unique()

array([ 9.37,  9.26,  9.25,  9.17,  9.16,  9.15,  9.13,  9.11,  9.1 ,
        9.06,  9.05,  9.04,  8.98,  8.93,  8.92,  8.88,  8.84,  8.83,
        8.82,  8.81,  8.8 ,  8.78,  8.77,  8.76,  8.75,  8.74,  8.73,
        8.72,  8.71,  8.69,  8.68,  8.67,  8.66,  8.65,  8.64,  8.62,
        8.61,  8.6 ,  8.59,  8.58,  8.57,  8.56,  8.55,  8.54,  8.53,
        8.52,  8.51,  8.5 ,  8.49,  8.48,  8.47,  8.46,  8.45,  8.44,
        8.43,  8.42,  8.41,  8.4 ,  8.39,  8.38,  8.37,  8.36,  8.35,
        8.34,  8.33,  8.32,  8.31,  8.3 ,  8.29,  8.28,  8.27,  8.26,
        8.25,  8.24,  8.23,  8.22,  8.21,  8.2 ,  8.19,  8.18,  8.17,
        8.16,  8.15,  8.14,  8.13,  8.12,  8.11,  8.1 ,  8.09,  8.08,
        8.07,  8.06,  8.05,  8.04,  8.03,  8.02,  8.01,  8.  ,  7.99,
        7.98,  7.97,  7.96,  7.95,  7.94,  7.93,  7.92,  7.91,  7.9 ,
        7.89,  7.88,  7.87,  7.86,  7.85,  7.84,  7.83,  7.82,  7.81,
        7.8 ,  7.79,  7.78,  7.77,  7.76,  7.75,  7.74,  7.73,  7.72,
        7.71,  7.7 ,

In [22]:
len(df.rating.unique())

599

In [23]:
df.groupby('name')['rating'].mean().sort_values(ascending=False)[:5]

,rating
name,
Taka no Tsume 8: Yoshida-kun no X-Files,10.00
Spoon-hime no Swing Kitchen,9.60
Mogura no Motoro,9.50
Kimi no Na wa.,9.37
Kahei no Umi,9.33


In [5]:
df.isnull().sum()

,0
anime_id,0
name,0
genre,62
type,25
episodes,0
rating,230
members,0


In [6]:
df['type']=df['type'].fillna('type')
print(df['type'].isnull().sum())
df['type']

0


,type
0,Movie
1,TV
2,TV
3,TV
4,TV
...,...
12289,OVA
12290,OVA
12291,OVA
12292,OVA


In [7]:
df['genre'] = df['genre'].fillna('unkown')
print(df['genre'].isnull().sum())
df['genre']

0


,genre
0,"Drama, Romance, School, Supernatural"
1,"Action, Adventure, Drama, Fantasy, Magic, Mili..."
2,"Action, Comedy, Historical, Parody, Samurai, S..."
3,"Sci-Fi, Thriller"
4,"Action, Comedy, Historical, Parody, Samurai, S..."
...,...
12289,Hentai
12290,Hentai
12291,Hentai
12292,Hentai


In [11]:
# Assuming df is your original DataFrame before any modifications to 'genre'
df['genre'] = df['genre'].fillna('unkown')  # Fill missing values if needed

# Check if 'genre' column contains strings
if df['genre'].apply(lambda x: isinstance(x, str)).all():
    df['genre'] = df['genre'].str.split(', ')
else:
    print("Warning: 'genre' column does not contain only strings. Skipping split operation.")
    # You might want to investigate the data types in 'genre' column further
    print(df['genre'].apply(type).value_counts())

In [12]:
df.describe()

,anime_id,rating,members
count,12294.000000,12064.000000,1.229400e+04
mean,14058.221653,6.473902,1.807134e+04
std,11455.294701,1.026746,5.482068e+04
min,1.000000,1.670000,5.000000e+00
25%,3484.250000,5.880000,2.250000e+02
50%,10260.500000,6.570000,1.550000e+03
75%,24794.500000,7.180000,9.437000e+03
max,34527.000000,10.000000,1.013917e+06


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12294 entries, 0 to 12293
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12294 non-null  int64  
 1   name      12294 non-null  object 
 2   genre     12294 non-null  object 
 3   type      12294 non-null  object 
 4   episodes  12294 non-null  object 
 5   rating    12064 non-null  float64
 6   members   12294 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.5+ KB


In [14]:
mlb=MultiLabelBinarizer()
mlb

MultiLabelBinarizer()

In [15]:
genres_encoded=mlb.fit_transform(df['genre'])
pd.DataFrame(genres_encoded)

,0
0,1
1,1
2,1
3,1
4,1
...,...
12289,1
12290,1
12291,1
12292,1


In [ ]:
broadcast_encoded=pd.get_dummies(df['anime_id'],prefix='broadcast')
broadcast_encoded

,broadcast_1,broadcast_5,broadcast_6,broadcast_7,broadcast_8,broadcast_15,broadcast_16,broadcast_17,broadcast_18,broadcast_19,...,broadcast_34498,broadcast_34501,broadcast_34502,broadcast_34503,broadcast_34506,broadcast_34514,broadcast_34519,broadcast_34522,broadcast_34525,broadcast_34527
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12289,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12290,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12291,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12292,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
scaler=MinMaxScaler()
scaler

MinMaxScaler()

In [ ]:
df['rating_scaled']=scaler.fit_transform(df[['rating']])
df['rating_scaled']

0        0.924370
1        0.911164
2        0.909964
3        0.900360
4        0.899160
           ...   
12289    0.297719
12290    0.313325
12291    0.385354
12292    0.397359
12293    0.454982
Name: rating_scaled, Length: 12294, dtype: float64

In [ ]:
df['community_members']=scaler.fit_transform(df[['members']])
df['community_members']

0        0.197872
1        0.782770
2        0.112689
3        0.664325
4        0.149186
           ...   
12289    0.000203
12290    0.000176
12291    0.000211
12292    0.000168
12293    0.000135
Name: community_members, Length: 12294, dtype: float64

In [ ]:
features=np.hstack((genres_encoded,broadcast_encoded.values,df[['rating_scaled', 'community_members']].values))
print(features)

[[1.00000000e+00 1.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  9.24369748e-01 1.97872202e-01]
 [1.00000000e+00 1.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  9.11164466e-01 7.82770102e-01]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 ... 0.00000000e+00
  9.09963986e-01 1.12689267e-01]
 ...
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  3.85354142e-01 2.11063682e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  3.97358944e-01 1.67667411e-04]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  4.54981993e-01 1.35120208e-04]]


In [24]:
### Cosine Similarity

In [25]:
df1= df.pivot_table(index='name',columns='anime_id',values='rating')

In [26]:
df1.fillna(0,axis=1,inplace=True)

In [27]:
df1

anime_id,1,5,6,7,8,15,16,17,18,19,...,34412,34447,34453,34464,34475,34476,34490,34503,34514,34519
name,,,,,,,,,,,,,,,,,,,,,
&quot;0&quot;,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"&quot;Aesop&quot; no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
&quot;Bungaku Shoujo&quot; Memoire,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
&quot;Bungaku Shoujo&quot; Movie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xxxHOLiC Movie: Manatsu no Yoru no Yume,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xxxHOLiC Rou,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
xxxHOLiC Shunmuki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [28]:
cosine_sim=cosine_similarity(df1)

In [29]:
cosine_sim[1]

array([0., 1., 0., ..., 0., 0., 0.])

In [30]:
df1.index

Index(['&quot;0&quot;',
       '&quot;Aesop&quot; no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu',
       '&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi',
       '&quot;Bungaku Shoujo&quot; Memoire',
       '&quot;Bungaku Shoujo&quot; Movie', '&quot;Eiji&quot;',
       '&quot;Eiyuu&quot; Kaitai', '.hack//G.U. Returner',
       '.hack//G.U. Trilogy', '.hack//G.U. Trilogy: Parody Mode',
       ...
       'on-chan, Yume Power Daibouken!', 's.CRY.ed', 'vivi', 'xxxHOLiC',
       'xxxHOLiC Kei', 'xxxHOLiC Movie: Manatsu no Yoru no Yume',
       'xxxHOLiC Rou', 'xxxHOLiC Shunmuki', 'Üks Uks', '◯'],
      dtype='object', name='name', length=12062)

In [49]:
def recommend_anime(similar_anime):
    if similar_anime in df1.index:
        index= np.where(similar_anime==df1.index)[0][0]
        similar= sorted(list(enumerate(cosine_sim[index])),key=lambda x: x[1],reverse=True)[1:6]#enumerate will provide the index and similarity score and we assign it in ascending order and only take the values from
        print(f'Recommended anime of {similar_anime}')
        print('-'*30)
        for movie in similar:
            print(df1.index[movie[0]])
    else:
        print('Anime is not in the list')

In [50]:
np.where('xxxHOLiC Shunmuki'==df1.index)

(array([12059]),)

In [51]:
recommend_anime('xxxHOLiC Shunmuki')

Recommended anime of xxxHOLiC Shunmuki
------------------------------
&quot;0&quot;
&quot;Aesop&quot; no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi
&quot;Bungaku Shoujo&quot; Memoire
&quot;Bungaku Shoujo&quot; Movie


In [52]:
similar=sorted(list(enumerate(cosine_sim[1])),reverse=True,key=lambda x: x[1])[1:6]
similar

[(0, 0.0), (2, 0.0), (3, 0.0), (4, 0.0), (5, 0.0)]

In [54]:
for movie in similar:
    print(df1.index[movie[0]])

&quot;0&quot;
&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi
&quot;Bungaku Shoujo&quot; Memoire
&quot;Bungaku Shoujo&quot; Movie
&quot;Eiji&quot;


In [55]:
####item_based

In [56]:
df2= df.pivot_table(index='anime_id',columns='name',values='rating')
df2.fillna(0,axis=1,inplace=True)

In [57]:
df2.head()

name,&quot;0&quot;,"&quot;Aesop&quot; no Ohanashi yori: Ushi to Kaeru, Yokubatta Inu",&quot;Bungaku Shoujo&quot; Kyou no Oyatsu: Hatsukoi,&quot;Bungaku Shoujo&quot; Memoire,&quot;Bungaku Shoujo&quot; Movie,&quot;Eiji&quot;,&quot;Eiyuu&quot; Kaitai,.hack//G.U. Returner,.hack//G.U. Trilogy,.hack//G.U. Trilogy: Parody Mode,...,"on-chan, Yume Power Daibouken!",s.CRY.ed,vivi,xxxHOLiC,xxxHOLiC Kei,xxxHOLiC Movie: Manatsu no Yoru no Yume,xxxHOLiC Rou,xxxHOLiC Shunmuki,Üks Uks,◯
anime_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
cos_user=cosine_similarity(df2)

In [61]:
# Get unique anime IDs from df2's index instead of df
anime_ids = df2.index.unique()

# Create df3 using the correct anime IDs
df3 = pd.DataFrame(cos_user, index=anime_ids, columns=anime_ids)

df3.head()

anime_id,1,5,6,7,8,15,16,17,18,19,...,34412,34447,34453,34464,34475,34476,34490,34503,34514,34519
anime_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [62]:
df3.shape

(12064, 12064)

In [64]:
thershold = 0.5
user = 7
# Assuming you want to drop the row with anime_id 3
new = df3.drop(index=3, errors='ignore')  # Use errors='ignore' to avoid KeyError if 3 is not found
# OR, drop the row by position
#new = df3.drop(df3.index[2]) # drop the 3rd row
new


anime_id,1,5,6,7,8,15,16,17,18,19,...,34412,34447,34453,34464,34475,34476,34490,34503,34514,34519
anime_id,,,,,,,,,,,,,,,,,,,,,
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34476,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
34490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
34503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [66]:
# Instead of using a hardcoded '3', use an existing anime_id from your DataFrame
anime_id_to_check = df3.index[2]  # Get the 3rd anime ID, for example

# Replace '3' with the actual anime ID in your code
filtered_new = new[new[anime_id_to_check] > 0.5]
result = filtered_new[anime_id_to_check].sort_values(ascending=False)[:10]

print(result)

anime_id
6    1.0
Name: 6, dtype: float64


In [68]:
df[(df.anime_id==6) | (df.anime_id==6100)]

,anime_id,name,genre,type,episodes,rating,members
214,6,Trigun,[unkown],TV,26,8.32,283069


In [73]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

# Step 1: Simulate User-Item Interactions
# Create a fake user-item interaction matrix based on ratings
np.random.seed(42)
num_users = 500  # Simulate 500 users

# Generate user preferences randomly based on genres and ratings
user_ids = [f"User_{i}" for i in range(1, num_users + 1)]
anime_ids = df['anime_id'].tolist()

# Create a user-item interaction matrix
interaction_matrix = {
    user: {
        anime: np.random.choice([0, 1], p=[0.8, 0.2]) if row['rating'] >= 7 else 0
        for anime, row in df.iterrows()
    }
    for user in user_ids
}

# Convert interaction matrix to a DataFrame
interaction_df = pd.DataFrame.from_dict(interaction_matrix, orient='index').fillna(0).astype(int)

# Step 2: Split Data into Train and Test Sets
def train_test_split_interaction(interaction_df, test_size=0.2):
    train_data = interaction_df.copy()
    test_data = interaction_df.copy()

    for user in interaction_df.index:
        # Get all interactions for the user
        user_interactions = interaction_df.loc[user]
        interacted_items = user_interactions[user_interactions > 0].index.tolist()

        # Randomly select test items
        test_items = np.random.choice(interacted_items, size=int(len(interacted_items) * test_size), replace=False)
        train_data.loc[user, test_items] = 0  # Remove test items from train
        test_data.loc[user, [item for item in interacted_items if item not in test_items]] = 0

    return train_data, test_data

train_data, test_data = train_test_split_interaction(interaction_df)

# Step 3: Generate Recommendations
def generate_recommendations(train_data, top_k=10):
    # Recommend top_k items based on global popularity (sum of interactions)
    item_popularity = train_data.sum(axis=0).sort_values(ascending=False)
    recommendations = {}

    for user in train_data.index:
        seen_items = train_data.loc[user][train_data.loc[user] > 0].index.tolist()
        recommended_items = [
            item for item in item_popularity.index if item not in seen_items
        ][:top_k]
        recommendations[user] = recommended_items

    return recommendations

recommendations = generate_recommendations(train_data)

# Step 4: Evaluate the Recommendation System
def evaluate_recommendations(recommendations, test_data):
    precision_list = []
    recall_list = []

    for user, recommended_items in recommendations.items():
        # Get actual relevant items from the test set
        actual_items = test_data.loc[user][test_data.loc[user] > 0].index.tolist()

        # Calculate precision and recall for the user
        if actual_items:
            true_positives = len(set(recommended_items) & set(actual_items))
            precision = true_positives / len(recommended_items) if recommended_items else 0
            recall = true_positives / len(actual_items) if actual_items else 0

            precision_list.append(precision)
            recall_list.append(recall)

    # Calculate average precision, recall, and F1-score
    avg_precision = np.mean(precision_list)
    avg_recall = np.mean(recall_list)
    f1 = (2 * avg_precision * avg_recall) / (avg_precision + avg_recall) if (avg_precision + avg_recall) > 0 else 0

    return avg_precision, avg_recall, f1

avg_precision, avg_recall, f1 = evaluate_recommendations(recommendations, test_data)

print(f"Precision: {avg_precision:.2f}")
print(f"Recall: {avg_recall:.2f}")
print(f"F1-Score: {f1:.2f}")


Precision: 0.05
Recall: 0.00
F1-Score: 0.01




---



#### Interview Questions:

#1. Can you explain the difference between user-based and item-based collaborative filtering?
User-Based Collaborative Filtering

Focus: 
Compares users to identify similar preferences.

Key Idea:
Users who have shown similar behavior (e.g., rated items similarly or purchased similar products) are likely to have similar preferences in the future.

Approach:
1.Calculate similarity between users based on their interactions (e.g., using cosine similarity, Pearson correlation, etc.).
2.Recommend items that similar users have interacted with but the current user has not.
 Example: 
If User A and User B both like movies X and Y, and User B likes movie Z, the system may recommend movie Z to User A.

Strengths:
1.Effective in domains with many overlapping user preferences
2.Leverages group behavior to predict individual preferences.

Weaknesses:
1.Struggles with sparse data (when users have interacted with only a few items)
2.Computationally expensive for large datasets as it involves comparing all users.

Item-Based Collaborative Filtering 

Focus:
Compares items to identify similar characteristics.

Key Idea: 
Items that are rated or interacted with similarly by users are likely to be similar in nature.

Approach:
1.Calculate similarity between items based on user interactions (e.g., ratings or purchase history). 
2.Recommend items similar to those the user has already interacted with

Example: 
If many users who bought product A also bought product B, and a new user buys product A, the system may recommend product B.

Strengths:
1.Works well in domains where user behavior is sparse, as it focuses on items rather than users.
2.More stable over time since item relationships are less dynamic than user relationships.
3.Efficient for large datasets due to the relatively smaller number of items compared to users.

Weaknesses:
1.Struggles with cold start for new items, as there’s no historical data to determine similarity.
2.May not capture unique, individual preferences as effectively as user-based approaches.
                    

#2.What is collaborative filtering, and how does it work?
Collaborative filtering is a technique used in recommendation systems that suggests items to users based on their past interactions and the behavior of other users. It operates under the assumption that users who have had similar preferences in the past will continue to do so in the future.
Types of Collaborative Filtering
1.Memory-Based Collaborative Filtering
Directly uses the user-item interaction matrix.
Divided into two subtypes:
User-Based: Finds similar users and recommends items they liked.
Item-Based: Finds similar items and recommends those similar to items the user interacted with.
How It Works:
Measure similarity (e.g., using cosine similarity, Pearson correlation).
Make predictions based on neighbors (similar users or items).

2. Model-Based Collaborative Filtering
Uses machine learning techniques to model user-item interactions.
Builds a predictive model from historical data, such as using
Neural Networks (e.g., Autoencoders)
Clustering or classification models
How It Works:
Learns latent factors (hidden relationships) from the user-item matrix.
Predicts ratings or interactions for new user-item pairs.

Steps in Collaborative Filtering
1.Data Collection:
Gather interaction data (e.g., ratings, purchases, views) to form a user-item matrix.
2.Similarity Calculation:
Compute similarity between users (user-based) or items (item-based) using metrics like:
3.Prediction:
Predict missing interactions in the user-item matrix by leveraging the interactions of similar users or items.
4.Recommendation:
Recommend top-ranked items with the highest predicted ratings or interaction likelihood.
Advantages
Domain Independence: Does not require knowledge of item characteristics.
Adaptability: Learns from user behavior dynamically.
Diversity: Can uncover unexpected recommendations through collaborative patterns.